# Data Preprocessing

In [1]:
%pip install azure-storage-blob
%pip install pandas
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from azure.storage.blob import BlobServiceClient
import pandas as pd
from io import StringIO
from sklearn.preprocessing import StandardScaler

# Extracting data from Azure Blob Storage

In [3]:
connection_string = "DefaultEndpointsProtocol=https;AccountName=telcodataset;AccountKey=8ZEMJBc+ZvMau0c/fOB82xUUQuK6Zy70bDbdV6McaZlub8yx6Na5Lm/PljqgfF84tR/kAKSXE/GO+AStiQUIEQ==;EndpointSuffix=core.windows.net"
container_name = "telco-container"
blob_name = "Telco-Customer-Churn-Dataset.csv"

In [4]:
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

In [6]:
download_stream = blob_client.download_blob()
csv_data = download_stream.readall()

HttpResponseError: The specified account is disabled.
RequestId:55de41c5-801e-007d-7e1e-5aede4000000
Time:2024-12-29T18:19:14.6509884Z
ErrorCode:AccountIsDisabled
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>AccountIsDisabled</Code><Message>The specified account is disabled.
RequestId:55de41c5-801e-007d-7e1e-5aede4000000
Time:2024-12-29T18:19:14.6509884Z</Message></Error>

In [7]:
data = pd.read_csv(StringIO(csv_data.decode('utf-8')))
print("Original data:")
print(data.head())

NameError: name 'csv_data' is not defined

Next, we convert 'TotalCharges' to numeric (forcing errors to NaN), fill missing values with the median and assign the result back to the column

In [47]:
data['TotalCharges'] = pd.to_numeric(data['TotalCharges'], errors='coerce')
data['MonthlyCharges'] = data['MonthlyCharges'].fillna(data['MonthlyCharges'].median())
data['tenure'] = data['tenure'].fillna(data['tenure'].median())
data['TotalCharges'] = data['TotalCharges'].fillna(data['TotalCharges'].median())

Then, we impute missing values for categorical columns with mode (most frequent value).

In [48]:
data['Churn'] = data['Churn'].fillna(data['Churn'].mode()[0])

Let's verify if the missing values have been handeled correctly: 

In [49]:
print("\nData after handling missing values:")
print(data.isnull().sum())


Data after handling missing values:
customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64


### Binning Tenure Values

In this step, we are categorizing the continuous `tenure` values (which represent the number of months a customer has been with the company) into discrete bins. This process is called **binning** or **discretization**, and it helps in transforming continuous data into categorical data, which can be more useful for certain types of analysis or modeling.

The bins are defined as follows:

- 0 to 12 months (0-1 Year)
- 12 to 24 months (1-2 Years)
- 24 to 36 months (2-3 Years)
- 36 to 48 months (3-4 Years)
- 48 to 60 months (4-5 Years)
- 60 to 72 months (5+ Years)

We are using the `pd.cut()` function to assign each `tenure` value to one of these bins. The `right=True` argument ensures that the bins are right-inclusive, meaning that a `tenure` value of exactly 12 months would be categorized as "1-2 Years". 

After applying this transformation, we will check for any missing values in the new `TenureGroup` column to ensure that all data has been successfully assigned to one of the bins.


In [50]:
bins = [-1, 12, 24, 36, 48, 60, 72]  
labels = ['0-1 Year', '1-2 Years', '2-3 Years', '3-4 Years', '4-5 Years', '5+ Years']  

data['TenureGroup'] = pd.cut(data['tenure'], bins=bins, labels=labels, right=True)  

print(data['TenureGroup'].isnull().sum())  # This should return 0 if no NaN values

# Verify the grouping
print(data[['tenure', 'TenureGroup']].head())


0
   tenure TenureGroup
0       1    0-1 Year
1      34   2-3 Years
2       2    0-1 Year
3      45   3-4 Years
4       2    0-1 Year


In [51]:
print(data[data['TenureGroup'].isnull()])

Empty DataFrame
Columns: [customerID, gender, SeniorCitizen, Partner, Dependents, tenure, PhoneService, MultipleLines, InternetService, OnlineSecurity, OnlineBackup, DeviceProtection, TechSupport, StreamingTV, StreamingMovies, Contract, PaperlessBilling, PaymentMethod, MonthlyCharges, TotalCharges, Churn, TenureGroup]
Index: []

[0 rows x 22 columns]


In [62]:
data['tenure'].max()

np.int64(72)

In [53]:
# Add calculated column 'TotalCharges' (MonthlyCharges * tenure)
data['TotalCharges'] = data['MonthlyCharges'] * data['tenure']

In [54]:
# Check the columns in the DataFrame to ensure 'Gender', 'InternetService', and 'Contract' exist
print(data.columns)

# Data Encoding
# 4. Data Encoding: Convert categorical columns to numerical using One-Hot Encoding
# Ensure the specified columns exist in the DataFrame
if 'Gender' in data.columns and 'InternetService' in data.columns and 'Contract' in data.columns:
    data = pd.get_dummies(data, columns=['Gender', 'InternetService', 'Contract'])
else:
    print("One or more specified columns are not in the DataFrame.")

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn',
       'TenureGroup'],
      dtype='object')
One or more specified columns are not in the DataFrame.


In [55]:
scaler = StandardScaler()
data['MonthlyCharges_scaled'] = scaler.fit_transform(data[['MonthlyCharges']])


In [56]:
# 6. Remove Outliers: Detect and remove outliers based on IQR for 'MonthlyCharges'
Q1 = data['MonthlyCharges'].quantile(0.25)
Q3 = data['MonthlyCharges'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

In [57]:
# Remove rows where 'MonthlyCharges' is outside the defined range
data = data[(data['MonthlyCharges'] >= lower_bound) & (data['MonthlyCharges'] <= upper_bound)]


In [58]:
# Save the processed data locally for review
data.to_csv("processed_data.csv", index=False)
print("\nData processed and saved locally as 'processed_data.csv'.")


Data processed and saved locally as 'processed_data.csv'.


In [59]:
data = pd.read_csv("processed_data.csv")
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   customerID             7043 non-null   object 
 1   gender                 7043 non-null   object 
 2   SeniorCitizen          7043 non-null   int64  
 3   Partner                7043 non-null   object 
 4   Dependents             7043 non-null   object 
 5   tenure                 7043 non-null   int64  
 6   PhoneService           7043 non-null   object 
 7   MultipleLines          7043 non-null   object 
 8   InternetService        7043 non-null   object 
 9   OnlineSecurity         7043 non-null   object 
 10  OnlineBackup           7043 non-null   object 
 11  DeviceProtection       7043 non-null   object 
 12  TechSupport            7043 non-null   object 
 13  StreamingTV            7043 non-null   object 
 14  StreamingMovies        7043 non-null   object 
 15  Cont

In [60]:
print(data.describe())

       SeniorCitizen       tenure  MonthlyCharges  TotalCharges  \
count    7043.000000  7043.000000     7043.000000   7043.000000   
mean        0.162147    32.371149       64.761692   2279.581350   
std         0.368612    24.559481       30.090047   2264.729447   
min         0.000000     0.000000       18.250000      0.000000   
25%         0.000000     9.000000       35.500000    394.000000   
50%         0.000000    29.000000       70.350000   1393.600000   
75%         0.000000    55.000000       89.850000   3786.100000   
max         1.000000    72.000000      118.750000   8550.000000   

       MonthlyCharges_scaled  
count           7.043000e+03  
mean           -6.204512e-17  
std             1.000071e+00  
min            -1.545860e+00  
25%            -9.725399e-01  
50%             1.857327e-01  
75%             8.338335e-01  
max             1.794352e+00  


In [61]:
# Count missing values
print(data.isnull().sum())

customerID               0
gender                   0
SeniorCitizen            0
Partner                  0
Dependents               0
tenure                   0
PhoneService             0
MultipleLines            0
InternetService          0
OnlineSecurity           0
OnlineBackup             0
DeviceProtection         0
TechSupport              0
StreamingTV              0
StreamingMovies          0
Contract                 0
PaperlessBilling         0
PaymentMethod            0
MonthlyCharges           0
TotalCharges             0
Churn                    0
TenureGroup              0
MonthlyCharges_scaled    0
dtype: int64
